# Explainability SHAP — CarVision Market Intelligence

Este notebook muestra un ejemplo de análisis SHAP (global y local) sobre el modelo de pricing entrenado en CarVision.

Supone que ya has ejecutado el pipeline de entrenamiento ("make train") y que el modelo está disponible en `artifacts/model.joblib`.


In [ ]:
import json
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import shap
import yaml

# Cargar configuración
config_path = Path("configs/config.yaml")
cfg = yaml.safe_load(config_path.read_text())
paths = cfg["paths"]

# Cargar datos y modelo
data_path = Path(paths["data_path"])
model_path = Path(paths["model_path"])

df = pd.read_csv(data_path)
model = joblib.load(model_path)

# Seleccionar un subconjunto de filas para el análisis SHAP
feature_cols = [c for c in df.columns if c != cfg["training"]["target"]]
X = df[feature_cols]
X_sample = X.sample(n=min(200, len(X)), random_state=42)
X_explain = X_sample.copy()
X_explain.head()


In [ ]:
# Crear explainer para el RandomForestRegressor dentro del Pipeline
# Asumimos que el modelo es un Pipeline sklearn con el estimador final en 'regressor' o 'model'.
from sklearn.pipeline import Pipeline

if isinstance(model, Pipeline):
    final_est = model[-1]
else:
    final_est = model

explainer = shap.TreeExplainer(final_est)
# Importante: trabajamos con las features transformadas por el Pipeline
X_transformed = model[:-1].transform(X_explain) if isinstance(model, Pipeline) else X_explain

shap_values = explainer.shap_values(X_transformed)
shap_values[:5]


## SHAP global — summary plot

Este gráfico muestra la importancia global de las features en el modelo de pricing.


In [ ]:
# Para visualizar correctamente en notebooks
shap.initjs()

shap.summary_plot(shap_values, X_transformed, show=False)


## SHAP local — explicación de un vehículo específico

Seleccionamos un vehículo del conjunto de explicación y mostramos su explicación local mediante force plot.


In [ ]:
idx = 0
x_single = X_transformed[idx:idx+1]

shap.force_plot(
    explainer.expected_value,
    shap_values[idx],
    x_single,
    matplotlib=True
)


## Notas

- Este notebook es de demostración: no se usa en el pipeline de producción.
- Ajusta el tamaño de la muestra (`n`) y las columnas de features según tus necesidades.
- Asegúrate de tener instalada la librería `shap` en tu entorno.
